In [1]:
import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

In [2]:
N = 1000
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)

In [165]:
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3, padding = 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding = 2)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        print('1', x.size())
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        print('2', x.size())
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        print('3', x.size())
        x = F.relu(self.fc1(x.view(-1, 256)))
        print('4', x.size())
        x = self.fc2(x)
        print('5', x.size())
        return F.log_softmax(x, dim = 1)

In [213]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        nb_hidden = 200
        self.conv1 = nn.Conv2d(2, 32, kernel_size=5, padding = 3)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5, padding = 3)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=2)
        self.fc1 = nn.Linear(1024, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        #print(x.size())
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        #print(x.size())
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        #print(x.size())
        x = F.relu(self.conv3(x))
        #print(x.size())
        x = F.relu(self.fc1(x.view(-1, 1024)))
        #print(x.size())
        x = self.fc2(x)
        #print(x.size())
        return F.log_softmax(x, dim = 1)

In [214]:
def train_model(model, train_input, train_target):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-2)
    nb_epochs = 25

    for e in range(nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()
        nb_train_errors = compute_nb_errors(model, train_input, train_target)
        print('train_error', nb_train_errors)
        print('loss', loss.item())

def compute_nb_errors(model, data_input, data_target):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output.data, 1)
        for k in range(mini_batch_size):
            if data_target.data[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1
    return nb_data_errors

In [215]:
train_input, train_target = Variable(train_input), Variable(train_target)
test_input, test_target = Variable(test_input), Variable(test_target)

mini_batch_size = 100

for k in range(1):
    model = Net2()
    train_model(model, train_input, train_target)
    nb_train_errors = compute_nb_errors(model, train_input, train_target) / train_input.size(0) * 100,
    nb_test_errors = compute_nb_errors(model, test_input, test_target)
    print('train error', nb_train_errors)
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                      nb_test_errors, test_input.size(0)))

train_error 462
loss 0.6815496683120728


KeyboardInterrupt: 